In [ ]:
import requests as req
from lxml import etree
import numpy as np
import time
import re

headers = {
    # 在浏览器中，network查看
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36 Edg/103.0.1264.62',
    'Cookie': '',
    'Connection':'close',
}


## 版面页

In [ ]:
urls = []  # 保存页面uid
limit = 5  # 版面页数，请勿设置过多

for i in range(1,limit+1):  # 获取近期回复前limit页中的帖子地址
    
    time.sleep(1) 
    
    mainPage = req.get('https://bbs.nga.cn/thread.php?fid=650&order_by=lastpostdesc&page='+str(i),headers=headers,verify=False)
    doc = etree.HTML(mainPage.text)
    pages_url = doc.xpath('//td[@class="c1"]/a')
    
    for pg in pages_url:
        r = re.search(r'[0-9]+',pg.attrib['href']).group()   # 帖子uid
        urls.append(r)
        print('no.'+str(i)+' : '+str(r))


In [ ]:
urls = set(urls) # 帖子去重,注意此处顺序被打乱
urls = list(urls)
print(len(urls))

## 主题页

In [ ]:
uid = [] 
for item in  urls:  # 帖子中用户uid获取
    
    time.sleep(1)
    
    page_url = 'https://bbs.nga.cn/read.php?tid='+str(item)+'&lite=js'         # 获取当前帖子页数
    mainPage = req.get(page_url,headers=headers,verify=False)
    txt = str(mainPage.text).replace('window.script_muti_get_var_store=','')
    Rows = re.findall(r'"__ROWS"\:[0-9]+',txt)
    
    if Rows:        # nga一小部分帖子js只传一半
         pass
    else:
        continue
        
    pageNum = int(int(Rows[0].replace('"__ROWS":',''))/20 + 1)  # 当前帖子页数
    print(str(item)+" pages: "+str(pageNum))
    
    if pageNum>100:  #去除超过100页的帖子
        continue
    
    for i in range(1,pageNum+1):       # 用户uid获取
        u = page_url+'&page='+str(i)
        mainPage = req.get(u,headers=headers,verify=False)
        txt = str(mainPage.text).replace('window.script_muti_get_var_store=','')
        tmp = re.findall(r'"uid"\:[0-9]+',txt)
        flag = 0
        for t in tmp:
            if(flag==0):
                flag=1
                continue
            i = t.replace('"uid":','')
            uid.append(i)
uid

In [ ]:
len(uid)  # 用户人数，不去重则更能反应活跃度，去重则反应真实人数

## 用户页

In [ ]:
url = 'https://bbs.nga.cn/nuke.php?lite=js&__lib=ucp&__act=get&uid='  # 用户IP查询
ips = [] # 保存ip
nums = 0
for person in uid:
    time.sleep(0.1)
    person_page_url = url + person
    mainPage = req.get(person_page_url,headers=headers,verify=False)
    txt = str(mainPage.text).replace('window.script_muti_get_var_store=','')
    tmp = re.findall(r'"ipLoc"\:"[\u4e00-\u9fa5]+',txt) # 正则查找
    if tmp:  # nga有概率js只传一半
        pass
    else:
        continue
    tmp = tmp[0].replace('"ipLoc":"','')
    ips.append(tmp)
    print(nums,tmp)  # 输出当前位置，方便网络中断后继续运行
    nums = nums+1

In [ ]:
# 中断续爬
# nums = 0
# for person in range(7705,9825):
#     time.sleep(0.1)
#     person_page_url = url + uid[person]
#     mainPage = req.get(person_page_url,headers=headers,verify=False)
#     txt = str(mainPage.text).replace('window.script_muti_get_var_store=','')
#     tmp = re.findall(r'"ipLoc"\:"[\u4e00-\u9fa5]+',txt) # 正则查找
#     if tmp:  # nga有概率js只传一半
#         pass
#     else:
#         continue
#     tmp = tmp[0].replace('"ipLoc":"','')
#     ips.append(tmp)
#     print(nums,tmp)
#     nums = nums+1
# len(ips)

In [ ]:
with open('.\\area.txt', mode='a',encoding='utf-8') as f:  # 写入文件保存
    for i in ips:
        f.write(i+'\n')

## 结果处理

In [ ]:
# 数据处理
data = np.array(ips)
data

In [ ]:
unique,count=np.unique(data,return_counts=True)  # 分类计数
data_count=dict(zip(unique,count))
data_count

In [ ]:
dict_sorted = sorted(data_count.items(),key=lambda x:x[1],reverse=True) # 排序
dict_sorted[:50]

In [ ]:
# 规范数据格式，方便导入echats
lists1= []
lists2= []
for i,j in dict_sorted:
    if j>=50: 
        lists1.append(i)
        lists2.append(j)

lists1,lists2  

In [ ]:
# 规范数据格式，方便导入echats
lists = []
for i,j in dict_sorted:
    x = {'value' : j,'name' : i}
    lists.append(x)
lists